In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#!pip install -U imbalanced-learn

In [0]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input, Masking
from keras.layers import Dense, Input, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM, SimpleRNN
from keras.layers import Embedding
import keras.backend as K 
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phraser, Phrases
import re 
from gensim.parsing.preprocessing import remove_stopwords
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [0]:
%%time
# reading the data
df = pd.read_csv('/content/drive/Shared drives/DSO 560 NLP Project/data.csv')
# creating a subset of the relevant attribute name and then dropping the column
df = df[df['attribute_name'] == 'style'].drop(columns = ['attribute_name'])
# combining similar category attribute values and removing spaces
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'businesscasual' if x == 'business casual' else x)

CPU times: user 416 ms, sys: 73.5 ms, total: 489 ms
Wall time: 546 ms


In [0]:
df.head()

,product_id,brand,product_full_name,description,brand_category,attribute_value,details
0,01DPGV4YRP3Z8J85DASGZ1Y99W,frame,les second medium noir,"minimal , modern styling meet refined luxury l...",accessory,casual,NaN
5,01DPH1DEN9G2WM7WAMJMD0A9W4,j crew,tie waist shirtdress stripe,take classic button silhouette turn ultra flat...,dressesandjumpsuits,casual,NaN
12,01E2KYW52BAG606GQ7A9H5R0KD,alo,interval microfleece pullover hoodie,articulate seam extra wide rib hem create shap...,unknown,casual,"xs 0 2 , s 4 6 , m 8 10 , l 12 14"
13,01DT513RRYT3SKH6X25G5VCH6B,chlo,leather ankle boot,heel measure approximately 55 mm 2 inch 30 mm ...,shoe boots ankle,androgynous,"fit small size , size large normal available s..."
20,01E2KM0KW6NB1JKMZVRXR6H8G2,alo,stadium quarter zip hoodie,supersoft hoodie design elastic hem cuff perfe...,unknown,casual,"xs 0 2 , s 4 6 , m 8 10 , l 12 14"


In [0]:
# creating a feature combining brand, productname, description and brand category
df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']).apply(str)

In [0]:
# the predictor 'text' is assigned to 'X' and the 'attribute_value' t 'y'
X = df['text'].values
# one-hot-encoding the y variable
y = pd.get_dummies(df['attribute_value'])
label_list = y.columns
y = y.values

In [0]:
# tokenizing the data and integer encoding it
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(X)

In [0]:
# running the tokenizer function on features
X = tokenizer.texts_to_sequences(X)

In [0]:
# getting vocab size and length for input
length = max([len(s.split()) for s in df['text']])
vocab_size = len(tokenizer.word_index) + 1

In [0]:
# padding the data to make it same size (max size)
X = pad_sequences(X, maxlen=length, padding='post')

In [0]:
# splitting training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [0]:
# balancing the dataset to account for disproportionate of labels
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [0]:
num_classes = y.shape[1]

In [0]:
# using glove vector to create function that makes word emebeddings
def load_glove_vectors():
    embeddings_index = {}
    with open('/content/drive/My Drive/NLP/glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

In [0]:
# creating an LSTM CNN model
def define_model():
    
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=length, trainable=False))
    #model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=64, kernel_size=10, padding='same', activation = 'relu'))
    model.add(Conv1D(filters=64, kernel_size=10, padding='same', activation = 'relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))

    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(num_classes))
    model.add(Activation('sigmoid'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [0]:
model = define_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 155, 100)          611400    
_________________________________________________________________
lstm_1 (LSTM)                (None, 155, 128)          117248    
_________________________________________________________________
lstm_2 (LSTM)                (None, 155, 128)          131584    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 77, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 77, 64)            81984     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 77, 64)            41024     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 38, 64)           

In [0]:
# fitting model on training data
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

Train on 20366 samples, validate on 5092 samples
Epoch 1/10
20366/20366 [==============================] - 46s 2ms/step - loss: 0.4694 - accuracy: 0.8960 - val_loss: 0.3636 - val_accuracy: 0.9091
Epoch 2/10
20366/20366 [==============================] - 40s 2ms/step - loss: 0.3016 - accuracy: 0.9091 - val_loss: 0.4353 - val_accuracy: 0.9091
Epoch 3/10
20366/20366 [==============================] - 42s 2ms/step - loss: 0.2958 - accuracy: 0.9091 - val_loss: 0.4327 - val_accuracy: 0.9091
Epoch 4/10
20366/20366 [==============================] - 43s 2ms/step - loss: 0.2951 - accuracy: 0.9091 - val_loss: 0.4349 - val_accuracy: 0.9091
Epoch 5/10
20366/20366 [==============================] - 40s 2ms/step - loss: 0.2945 - accuracy: 0.9091 - val_loss: 0.4271 - val_accuracy: 0.9091
Epoch 6/10
20366/20366 [==============================] - 41s 2ms/step - loss: 0.2942 - accuracy: 0.9091 - val_loss: 0.4343 - val_accuracy: 0.9091
Epoch 7/10
20366/20366 [==============================] - 42s 2ms/ste

In [0]:
# evaluating model
model.evaluate(X_test, y_test)

1089/1089 [==============================] - 3s 2ms/step


[0.301320913901999, 0.909090518951416]

In [0]:
# making predictions
results = model.predict(X)

In [0]:
results_df = pd.DataFrame()
results_mask = results > 0.1
for i in range(len(label_list)):
    results_df[label_list[i]] = results_mask[:,i]
    results_df[label_list[i]] = results_df[label_list[i]].apply(int)

In [0]:
results_df['sum']  = 0
for key in label_list:
    results_df['sum'] = results_df['sum'] + results_df[key]
(results_df['sum'] == 0).sum()

0

In [0]:
results_df = results_df.drop(columns = ['sum'])
for key in label_list:
    results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

results_df['attribute_value'] = ''
for key in label_list:
    results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
results_df.head()

,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,glam,modern,retro,romantic,attribute_value
0,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,,,,,"androgynous, athleisure, boho, businesscasual,..."
1,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,,,,,"androgynous, athleisure, boho, businesscasual,..."
2,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,,,,,"androgynous, athleisure, boho, businesscasual,..."
3,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,,,,,"androgynous, athleisure, boho, businesscasual,..."
4,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,,,,,"androgynous, athleisure, boho, businesscasual,..."


In [0]:
results_df['attribute_value'].value_counts()

androgynous, athleisure, boho, businesscasual, casual, classic, edgy          10884
androgynous, athleisure, boho, businesscasual, casual, classic, edgy, glam        3
Name: attribute_value, dtype: int64

In [0]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('model.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
def get_style(brand, product_full_name, description, details, brand_category):
    def clean_text(x):
        try:
            x = re.sub(r'<.*?>', '',x)
            x = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", x)
            x = re.sub(r"\'s", " \'s", x)
            x = re.sub(r"\'ve", " \'ve", x)
            x = re.sub(r"n\'t", " n\'t", x)
            x = re.sub(r"\'re", " \'re", x)
            x = re.sub(r"\'d", " \'d", x)
            x = re.sub(r"\'ll", " \'ll", x)
            x = re.sub(r",", " , ", x)
            x = re.sub(r"!", " ! ", x)
            x = re.sub(r"\(", "", x)
            x = re.sub(r"\)", "", x)
            x = re.sub(r"\?", "", x)
            x = re.sub(r"/", "", x)
            x = re.sub(r"\s{2,}", " ", x)
            return x.lower()
        except:
            return ''

    def lemmatizer(x):
        return ' '.join([token.lemma_ for token in nlp(x)])
        
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    with open('model.pickle', 'rb') as handle:
        model = pickle.load(handle)

    data = pd.DataFrame({'brand':brand,'product_full_name':product_full_name,'description':description,'details':details,'brand_category':brand_category},index=[0]) 
    df = data.copy()
    for key in df.columns:
        df[key] = df[key].apply(clean_text)
        df[key] = df[key].apply(remove_stopwords)
        df[key] = df[key].apply(lemmatizer)
        df[key] = df[key].apply(clean_text)
        df[key] = df[key].apply(remove_stopwords)
    df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category'] + ' ' + df['details']).apply(str)
    X = df['text'].values
    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    results = model.predict(X)
    results_df = pd.DataFrame()
    results_mask = results > 0.1
    for i in range(len(label_list)):
        results_df[label_list[i]] = results_mask[:,i]
        results_df[label_list[i]] = results_df[label_list[i]].apply(int)
    results_df['sum']  = 0
    for key in label_list:
        results_df['sum'] = results_df['sum'] + results_df[key]
    (results_df['sum'] == 0).sum()
    results_df = results_df.drop(columns = ['sum'])
    for key in label_list:
        results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

    results_df['attribute_value'] = ''
    for key in label_list:
        results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

    results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
    data['attribute_value'] = results_df['attribute_value']
    return data

In [0]:
brand = "frame"
product_full_name = "les second medium noir"
description = "'minimal , modern styling meet refined luxury les second caba tote craft exquisite leather , structured handbag equip adjustable high polish peg buttonhole double drop handle , center welt seam , detachable pouch , frame logo discreetly emboss'"
details = np.nan
brand_category = 'accessory'

In [0]:
get_style(brand, product_full_name, description, details, brand_category)

,brand,product_full_name,description,details,brand_category,attribute_value
0,frame,les second medium noir,"'minimal , modern styling meet refined luxury ...",NaN,accessory,"androgynous, athleisure, boho, businesscasual,..."
